# Neural Network Demo

By Grant R. Vousden-Dishington

This notebook is heavily based on two blogs **A Neural Network in 11 lines of Python** ([Part 1](http://iamtrask.github.io/2015/07/12/basic-python-network/) & [Part 2]()) and **Hinton's Droptout in 3 lines of Python**, both by [*@iamtrask*](iamtrask.github.io)

In [10]:
import numpy as np

# Sigmoid function: "squashes" numbers into probabilities
def nonlin(x, deriv=False):
    x = x * (1-x) if deriv else 1 / (1 + np.exp(-x))
    return x

# Input: 4 training examples, 3-dimensional (4 x 3)
nnInput = np.array([
        [0, 0, 1],
        [0, 1, 1],
        [1, 0, 1],
        [1, 1, 1]
    ])

## 2-layer network

In [11]:
# Output: expected classifications (4 x 1)
nnOutput2 = np.array([[0, 0, 1 ,1]]).T

# Initialize weights with mean of 0, seed random generator
np.random.seed(261015)
syn0 = 2 * np.random.random((3, 1)) - 1  # (3 x 1)

# Do full-batch training
for i in range(60000):
    
    # Forward propagation
    l0 = nnInput
    l1 = nonlin(l0 @ syn0)
    
    # Error: expected minus predicted (4 x 1)
    errl1 = nnOutput2 - l1
    
    # Calculate delta: the error times slope of the sigmoid at l1 values
    deltal1 = errl1 * nonlin(l1, deriv=True)
    
    # Update weights: matrix multiply the input times the delta values all at once
    syn0 += l0.T @ deltal1
    
print("Output after training")
print(l1)

Output after training
[[ 0.00390254]
 [ 0.00318154]
 [ 0.99740413]
 [ 0.99681544]]


## 3-layer network

In [12]:
# Output: expected classifications (4 x 1)
nnOutput3 = np.array([[0, 1, 1 ,0]]).T

# Initialize weights with mean of 0, seed random generator
np.random.seed(261015)
syn0 = 2 * np.random.random((3, 4)) - 1  # (3 x 4)
syn1 = 2 * np.random.random((4, 1)) - 1  # (4 x 1)

# Do full-batch learning
for i in range(60000):
    
    # Forward Propagation
    l0 = nnInput
    l1 = nonlin(l0 @ syn0)
    l2 = nonlin(l1 @ syn1)
    
    # Output error: expected minus predicted (4 x 1)
    errl2 = nnOutput3 - l2
    
    # Getting a running view of the error every 10000 steps
    if not i % 10000:
        print("Error: " + str(np.mean(np.abs(errl2))))
        
    # Calculate delta for output layer: the error times slope of the sigmoid at l2 values (4 x 1)
    deltal2 = errl2 * nonlin(l2, deriv=True)
    
    # Hidden error: how much hidden layer contributed to output error (4 x 4)
    errl1 = deltal2 @ syn1.T
    
    # Calculate delta for hidden layer: the error times the sigmoid at l2 values (4 x 4)
    deltal1 = errl1 * nonlin(l1, deriv=True)
    
    # Update weights
    syn0 += l0.T @ deltal1
    syn1 += l1.T @ deltal2

Error: 0.500646983197
Error: 0.0108313939135
Error: 0.00721546010071
Error: 0.00571646037978
Error: 0.00485302997199
Error: 0.00427728962702
